In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import sys

- 시퀀스 데이터(순차적으로 이루어진 데이터 / 대부분 문자로 이루어진 데이터/ 주로 문장들을 이해하는데 도움)
- 가장 활발하게 이용하는 부분이 챗봇 
- time series 
- POS tagging : 품사를 구분하기 위한 목적으로 사용
- BPTT 백프로포게이션을 시간에따라서 거슬러 올라가는 

#### 단일 cell 을 통한 계산 처리 과정 

In [3]:
# 입력 데이터 준비
inputs = np.array([[[1, 2]]])

In [165]:
import warnings
warnings.filterwarnings(action ="ignore")

tf.reset_default_graph()
tf.set_random_seed(777)
tf_inputs = tf.constant(inputs, dtype = tf.float32)

# RNN cell 만들기 (cell을 만들고 -> 어떻게 가동시킬지 설정)
# 1* 3의 matrix 출력([6, 7, 8]과 같은)
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=3) # 네모박스를 하나 만든 것
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, inputs= tf_inputs) 
# 네모박스를 실행 / 출력이 2번실행되기 때문에 2번 받을 수 있다. 
print(outputs)
print(state)
print("\n\n \eights")# 가중치 값
for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(w)
# 5행 3열인 이유 : Wxh 2행3열 과 Whh 3행3열 -> 5행3열 


Tensor("rnn/transpose_1:0", shape=(1, 1, 3), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(1, 3), dtype=float32)


 \eights
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


In [22]:
 with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        output_run, state_run = sess.run([outputs, state])
        print("Output Value")
        print(output_run)
        print("State Value")
        print(state_run)
        
        # output과 state의 값이 똑같다.
        
        variables_names = [v.name for v in tf.trainable_variables()]
        #print(variables_names) #하나는 가중치 하나는 바이어스
        values = sess.run(variables_names)
        for k, v in zip(variables_names, values):
            print(k, v) # 5행 3열짜리 가중치와 바이어스 

Output Value
[[[-0.9314169   0.75578666 -0.6819246 ]]]
State Value
[[-0.9314169   0.75578666 -0.6819246 ]]
rnn/basic_rnn_cell/kernel:0 [[-0.62831575  0.38538355  0.79733914]
 [-0.5203329   0.30046564 -0.8150209 ]
 [ 0.39399797  0.16670114  0.4062907 ]
 [-0.6391754   0.8460203   0.5266966 ]
 [ 0.41124135  0.66347724 -0.0210759 ]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


#### 다중 cell

In [32]:
# i work at google  ->  i = [[1, 0, 0, 0]] , work =[[0, 1, 0, 0]], at = [[0, 0, 1, 0]], google = [[0, 0, 0, 1]]
# i google at work ->    i = [[1, 0, 0, 0]] , google =[[0, 0, 0, 1]], at = [[0, 0, 1, 0]], work = [[0, 1, 0, 0]]

inputs = np.array([[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], # 입력데이터 1행 4열짜리가 순차적으로 들어가는  
                 [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]]]) 
print(inputs)

[[[1 0 0 0]
  [0 1 0 0]
  [0 0 1 0]
  [0 0 0 1]]

 [[1 0 0 0]
  [0 0 0 1]
  [0 0 1 0]
  [0 1 0 0]]]


In [36]:
tf.reset_default_graph()
tf.set_random_seed(777)
tf_inputs = tf.constant(inputs, dtype = tf.float32)

# cell 모델링
rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units = 3) # num_units 지정은 자유 -> 출력갯수 
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype = tf.float32, inputs = tf_inputs)
variables_names = [v.name for v in tf.trainable_variables()]
print(outputs) # 입력값개수, /입력되는 열의 갯수 / 출력 갯수 
print("--------------------------------")
print(state) # 여러 cell을 지나서 최종적으로 출력된 state : 전체 입력갯수 2문장, / 출력갯수
print("--------------------------------")
for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES): #
    print(w)

Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
--------------------------------
Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)
--------------------------------
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


In [38]:
 with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        output_run, state_run = sess.run([outputs, state])
        print("Output Value") # 단어가 4개이기 때문에 4개나옴 / 두개의 문장을 출력한것/ 열은 내가 임의로 지정한 것 n_units 
        # 4행 4열짜리가 4행 3열 되기위해서는 *4행 3열이 되야한다.
        # 처음 값은 똑같으나 가중치 계산이 되면서 계속 달라진다. 
        # 같은 단어여서 무조건 처음이 같은게 아니라 이전으로부터 입력받은 값 없이 순수한결과값이기 때문에 같은 건이다 (i 와 at 의 차이)
        print(output_run)
        print("State Value")
        # 처음 시작은 4행 3열로 시작하지만 마지막에는 2행 3열이 된다. 
        # 2행 3열이 3행 3열이 되기위해서는 *3형 3열이 되야한다.
        # 그래서 output의 최종결과값 하고 state는 값은 값이 출력된다.
        # 따라서 가중치는 4행 3열과 3행 3열의 합 7행 3열이 된다.
        print(state_run)
        
        # output과 state의 값이 똑같다.
        
        #print(variables_names) #하나는 가중치 하나는 바이어스
        values = sess.run(variables_names)
        for k, v in zip(variables_names, values):
            print(k, v) # 5행 3열짜리 가중치와 바이어스 

Output Value
[[[-0.50944704  0.33166462  0.6126557 ]
  [-0.20793891  0.24406303 -0.75278705]
  [-0.06346128 -0.52844936  0.68356085]
  [-0.36491966  0.8857268  -0.02324395]]

 [[-0.50944704  0.33166462  0.6126557 ]
  [-0.30707452  0.62735885  0.21719742]
  [ 0.5043804  -0.14038289  0.3744523 ]
  [-0.11641277  0.70696247 -0.7512605 ]]]
State Value
[[-0.36491966  0.8857268  -0.02324395]
 [-0.11641277  0.70696247 -0.7512605 ]]
rnn/basic_rnn_cell/kernel:0 [[-0.56198275  0.34469748  0.7131618 ]
 [-0.4653999   0.2687447  -0.7289769 ]
 [ 0.35240245  0.14910203  0.36339748]
 [-0.57169586  0.7567036   0.47109187]
 [ 0.3678255   0.5934322  -0.01885086]
 [ 0.31208777 -0.40880746  0.22867584]
 [ 0.5521256   0.682691   -0.5481483 ]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


In [39]:
inputs = np.array([[[1, 0, 0, 0]]])

In [40]:
tf.reset_default_graph()
tf.set_random_seed(777)
tf_inputs = tf.constant(inputs, dtype = tf.float32)

# cell 모델링
rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units = 3) # num_units 지정은 자유 -> 출력갯수 
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype = tf.float32, inputs = tf_inputs)
variables_names = [v.name for v in tf.trainable_variables()]
print(outputs) # 입력값개수, /입력될떄 열의 갯수 / 출력 갯수 
print("--------------------------------")
print(state) # 여러 cell을 지나서 최종적으로 출력된 state : 전체 입력갯수 2문장, / 출력갯수
print("--------------------------------")
for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES): #
    print(w)

Tensor("rnn/transpose_1:0", shape=(1, 1, 3), dtype=float32)
--------------------------------
Tensor("rnn/while/Exit_3:0", shape=(1, 3), dtype=float32)
--------------------------------
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


In [41]:
 with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        output_run, state_run = sess.run([outputs, state])
        print("Output Value") # 단어가 4개이기 때문에 4개나옴 / 두개의 문장을 출력한것/ 열은 내가 임의로 지정한 것 n_units 
        # 4행 4열짜리가 4행 3열 되기위해서는 *4행 3열이 되야한다.
        # 처음 값은 똑같으나 가중치 계산이 되면서 계속 달라진다. 
        # 같은 단어여서 무조건 처음이 같은게 아니라 이전으로부터 입력받은 값 없이 순수한결과값이기 때문에 같은 건이다 (i 와 at 의 차이)
        print(output_run)
        print("State Value")
        # 처음 시작은 4행 3열로 시작하지만 마지막에는 2행 3열이 된다. 
        # 2행 3열이 3행 3열이 되기위해서는 *3형 3열이 되야한다.
        # 그래서 output의 최종결과값 하고 state는 값은 값이 출력된다.
        # 따라서 가중치는 4행 3열과 3행 3열의 합 7행 3열이 된다.
        print(state_run)
        
        # output과 state의 값이 똑같다.
        
        #print(variables_names) #하나는 가중치 하나는 바이어스
        values = sess.run(variables_names)
        for k, v in zip(variables_names, values):
            print(k, v) # 5행 3열짜리 가중치와 바이어스 

Output Value
[[[-0.50944704  0.33166462  0.6126557 ]]]
State Value
[[-0.50944704  0.33166462  0.6126557 ]]
rnn/basic_rnn_cell/kernel:0 [[-0.56198275  0.34469748  0.7131618 ]
 [-0.4653999   0.2687447  -0.7289769 ]
 [ 0.35240245  0.14910203  0.36339748]
 [-0.57169586  0.7567036   0.47109187]
 [ 0.3678255   0.5934322  -0.01885086]
 [ 0.31208777 -0.40880746  0.22867584]
 [ 0.5521256   0.682691   -0.5481483 ]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


In [53]:
tf.reset_default_graph()
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

x_data = np.array([[h, e, l, l, o ]], dtype = np.float32)


rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units = 3)

outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype = tf.float32, inputs = x_data)

variables_names = [v.name for v in tf.trainable_variables()]
print(outputs) # 입력값개수, /입력될떄 열의 갯수 / 출력 갯수 
print("--------------------------------")
print(state) # 여러 cell을 지나서 최종적으로 출력된 state : 전체 입력갯수 2문장, / 출력갯수
print("--------------------------------")
print("--------------------------------")
for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES): #
    print(w)
    
with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        output_run, state_run = sess.run([outputs, state])
        print("Output Value") 
        print(output_run)
        print("State Value")
        print(state_run)

        values = sess.run(variables_names)
        for k, v in zip(variables_names, values):
            print(k, v)

Tensor("rnn/transpose_1:0", shape=(1, 5, 3), dtype=float32)
--------------------------------
Tensor("rnn/while/Exit_3:0", shape=(1, 3), dtype=float32)
--------------------------------
--------------------------------
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>
Output Value
[[[ 0.34300327 -0.107384   -0.46925384]
  [-0.2774119  -0.5878217   0.09722161]
  [ 0.75449204  0.33791998 -0.33005482]
  [-0.34582028 -0.25192338  0.33335426]
  [ 0.5475094  -0.22006126 -0.02506072]]]
State Value
[[ 0.5475094  -0.22006126 -0.02506072]]
rnn/basic_rnn_cell/kernel:0 [[ 0.35749233 -0.10779965 -0.5091131 ]
 [ 0.14107978 -0.518346    0.12631565]
 [ 0.41646826 -0.04292756 -0.00733286]
 [ 0.05838621 -0.5328102   0.13483173]
 [-0.41033363  0.37693584  0.38223803]
 [-0.6456835  -0.72144556  0.4305197 ]
 [ 0.7555529   0.7730297   0.24222267]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


- sequence_length : 입력데이터의 갯수 = 셀의 갯수
- hidden_size = 출력의 갯수 
- batch_size = 데이터의 갯수(훈련횟수)

#### hihello 학습 과정

In [126]:
idx2char = ['h', 'i', 'e','l','o']
x_one_hot = [[[1, 0, 0, 0, 0],
             [0, 1, 0, 0, 0],
             [1, 0, 0, 0, 0],
             [0, 0, 1, 0, 0],
             [0, 0, 0, 1, 0],
             [0, 0, 0, 1, 0],
             [0, 0, 0, 0, 1]]]
#x_data = [[0, 1, 0, 2, 3, 3]] # h는0, i는1, e는 2..... -> hihell
y_data = [[0, 1, 0, 2, 3, 3, 4]] # ihello 출력값

In [127]:
# parameter
sequence_len = 7
num_classes = 5
batch_size = 1

In [128]:
tf.reset_default_graph()
# X, y 준비 (입출력 데이터 준비 )
X = tf.placeholder(tf.float32, shape=[None, sequence_len, num_classes])  # 입력데이터는 3차원 (이미지만 4차원)/ 전체 데이터의 갯수, 입력되는 열의 갯수, 출력갯수
# 실제로는 출력갯수는 출력할 수 있는 갯수
y = tf.placeholder(tf.int32, shape=[None, sequence_len])  # y 값은 정수형으로 (원핫 인코딩했기 때문에)

In [129]:
# rnn 모델
cell  = tf.contrib.rnn.BasicRNNCell(num_units = num_classes)
outputs, state = tf.nn.dynamic_rnn(cell = cell, inputs = X, dtype = tf.float32)

print(outputs)
print(state)

Tensor("rnn/transpose_1:0", shape=(?, 7, 5), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(?, 5), dtype=float32)


In [130]:
# 평면화 (Flat Layer)
X_for_fc = tf.reshape(outputs, [-1, num_classes])
print(X_for_fc)

Tensor("Reshape:0", shape=(?, 5), dtype=float32)


In [119]:
# FC (W, b, logit)

outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc, num_outputs=num_classes, activation_fn=None) # 입력데이터, 출력의 갯수

outputs = tf.reshape(outputs, [batch_size, sequence_len, num_classes]) # 전체 데이터의 갯수, 데이터의 길이, 출력의 갯수
W = tf.ones([batch_size, sequence_len])
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y , weights=W)) # logits는 3차원으로
# 시퀀스 데이터이기 때문에 순서가 중요하다 / 일반적인 비용구하는 공식으로 하면 안된다.

train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        _, c = sess.run([train,cost], feed_dict={X:x_one_hot, y:y_data})
        result = sess.run(tf.argmax(outputs,2), feed_dict={X:x_one_hot})
        print(i, "cost :", c, "      prediction:", result, "    정답 :", y_data)
    result_str = [idx2char[c] for c in np.squeeze(result)]
    print("결과 :", ",".join(result_str) )

0 cost : 1.4880387       prediction: [[3 3 3 3 3 3]]     정답 : [[1, 0, 2, 3, 3, 4]]
1 cost : 1.3550959       prediction: [[3 3 3 3 3 3]]     정답 : [[1, 0, 2, 3, 3, 4]]
2 cost : 1.2024946       prediction: [[1 0 3 3 4 4]]     정답 : [[1, 0, 2, 3, 3, 4]]
3 cost : 1.010539       prediction: [[1 1 2 3 4 4]]     정답 : [[1, 0, 2, 3, 3, 4]]
4 cost : 0.8755426       prediction: [[1 1 2 3 3 4]]     정답 : [[1, 0, 2, 3, 3, 4]]
5 cost : 0.7909606       prediction: [[1 0 2 3 3 4]]     정답 : [[1, 0, 2, 3, 3, 4]]
6 cost : 0.6636115       prediction: [[1 0 2 3 3 4]]     정답 : [[1, 0, 2, 3, 3, 4]]
7 cost : 0.54720175       prediction: [[1 0 2 3 3 4]]     정답 : [[1, 0, 2, 3, 3, 4]]
8 cost : 0.47552872       prediction: [[1 0 2 3 3 4]]     정답 : [[1, 0, 2, 3, 3, 4]]
9 cost : 0.41703203       prediction: [[1 0 2 3 3 3]]     정답 : [[1, 0, 2, 3, 3, 4]]
10 cost : 0.35898462       prediction: [[1 0 2 3 3 4]]     정답 : [[1, 0, 2, 3, 3, 4]]
11 cost : 0.29545656       prediction: [[1 0 2 3 3 4]]     정답 : [[1, 0, 2, 3, 3, 4]

In [131]:
# FC (W, b, logit)

outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc, num_outputs=num_classes, activation_fn=None) # 입력데이터, 출력의 갯수

outputs = tf.reshape(outputs, [batch_size, sequence_len, num_classes]) # 전체 데이터의 갯수, 데이터의 길이, 출력의 갯수
W = tf.ones([batch_size, sequence_len])
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y , weights=W)) # logits는 3차원으로
# 시퀀스 데이터이기 때문에 순서가 중요하다 / 일반적인 비용구하는 공식으로 하면 안된다.

train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        _, c = sess.run([train,cost], feed_dict={X:x_one_hot, y:y_data})
        result = sess.run(tf.argmax(outputs,2), feed_dict={X:x_one_hot})
        print(i, "cost :", c, "      prediction:", result, "    정답 :", y_data)
    result_str = [idx2char[c] for c in np.squeeze(result)]
    print("결과 :", ",".join(result_str) )

0 cost : 1.7951401       prediction: [[3 4 3 4 3 3 0]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
1 cost : 1.3937826       prediction: [[0 1 0 4 3 3 0]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
2 cost : 1.1056939       prediction: [[0 1 0 1 3 3 0]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
3 cost : 0.8646723       prediction: [[0 1 0 1 3 3 0]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
4 cost : 0.69971305       prediction: [[0 1 0 1 3 3 4]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
5 cost : 0.54325426       prediction: [[0 1 0 1 3 3 4]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
6 cost : 0.42672417       prediction: [[0 1 0 2 3 3 4]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
7 cost : 0.34361717       prediction: [[0 1 0 2 3 3 4]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
8 cost : 0.2767791       prediction: [[0 1 0 2 3 3 4]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
9 cost : 0.2180494       prediction: [[0 1 0 2 3 3 4]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
10 cost : 0.16740768       prediction: [[0 1 0 2 3 3 4]]     정답 : [[0, 1, 0, 2, 3, 3, 4]]
11 cost : 0.12690246      

#### 두 번째 예제

In [205]:
sample = " if you want you"
idx2char = list(set(sample))
print(idx2char)

char2idx = {c:i for i, c in enumerate(idx2char)}
print(char2idx)

sample_idx = [char2idx[c] for c in sample]
print(sample_idx)

x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

['t', 'n', 'o', 'w', 'f', ' ', 'u', 'y', 'a', 'i']
{'t': 0, 'n': 1, 'o': 2, 'w': 3, 'f': 4, ' ': 5, 'u': 6, 'y': 7, 'a': 8, 'i': 9}
[5, 9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]


In [206]:
# parameter
num_classes = len(char2idx)
batch_szie = 1
sequence_len = len(sample) - 1 

In [207]:
tf.reset_default_graph()

X = tf.placeholder(tf.int32, shape=[None, sequence_len])  # one-hot인코딩을 사용할 것이기 때문에 2차원으로
y = tf.placeholder(tf.int32, shape=[None, sequence_len]) 

x_one_hot = tf.one_hot(X, num_classes)
print(x_one_hot)


Tensor("one_hot:0", shape=(?, 15, 10), dtype=float32)


In [208]:
cell  = tf.contrib.rnn.BasicRNNCell(num_units = num_classes)
outputs, state = tf.nn.dynamic_rnn(cell = cell, inputs = x_one_hot, dtype = tf.float32)

W = tf.ones([batch_size, sequence_len])
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y , wetights=W)) 
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)



In [209]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        _, c = sess.run([train,cost], feed_dict={X:x_data, y:y_data})
        result = sess.run(tf.argmax(outputs,2), feed_dict={X:x_data})
        print(i, "cost :", c, "      prediction:", result, "    정답 :", y_data)
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("결과 :", ",".join(result_str) )

0 cost : 2.2372687       prediction: [[6 1 5 6 2 6 5 9 5 1 5 6 5 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : u,n, ,u,o,u, ,i, ,n, ,u, ,o,u
1 cost : 1.8941674       prediction: [[9 1 5 6 2 6 5 9 5 1 0 5 6 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,n, ,u,o,u, ,i, ,n,t, ,u,o,u
2 cost : 1.7404529       prediction: [[9 1 5 7 2 6 5 6 8 1 0 5 7 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,n, ,y,o,u, ,u,a,n,t, ,y,o,u
3 cost : 1.5966445       prediction: [[9 1 5 7 5 6 5 3 8 1 0 5 7 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,n, ,y, ,u, ,w,a,n,t, ,y,o,u
4 cost : 1.4902977       prediction: [[9 1 5 7 5 6 5 3 8 1 0 5 7 5 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,n, ,y, ,u, ,w,a,n,t, ,y, ,u
5 cost : 1.4123538       prediction: [[9 1 5 7 2 6 5 3 8 1 0 5 7 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,n, ,y,o,u, ,w,a,n,t, ,y,o,u
6 cost : 1.3259403    

52 cost : 0.8211108       prediction: [[9 4 5 7 2 6 5 3 8 1 0 5 7 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,f, ,y,o,u, ,w,a,n,t, ,y,o,u
53 cost : 0.82070047       prediction: [[9 4 5 7 2 6 5 3 8 1 0 5 7 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,f, ,y,o,u, ,w,a,n,t, ,y,o,u
54 cost : 0.82024676       prediction: [[9 4 5 7 2 6 5 3 8 1 0 5 7 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,f, ,y,o,u, ,w,a,n,t, ,y,o,u
55 cost : 0.8197781       prediction: [[9 4 5 7 2 6 5 3 8 1 0 5 7 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,f, ,y,o,u, ,w,a,n,t, ,y,o,u
56 cost : 0.81940866       prediction: [[9 4 5 7 2 6 5 3 8 1 0 5 7 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,f, ,y,o,u, ,w,a,n,t, ,y,o,u
57 cost : 0.8190851       prediction: [[9 4 5 7 2 6 5 3 8 1 0 5 7 2 6]]     정답 : [[9, 4, 5, 7, 2, 6, 5, 3, 8, 1, 0, 5, 7, 2, 6]]
결과 : i,f, ,y,o,u, ,w,a,n,t, ,y,o,u
58 cost : 0.8

## LSTM (Long Shore Term Memory)

+ BackPropagation Through Time(BPTT)
    - Gradient Vanishing
    - Gradient Exploding
    
- memory cell : 새로운 출력이 하나 추가 

In [6]:
tf.reset_default_graph()
tf.set_random_seed(777)

inputs = np.array([[[1, 0]]])
tf_inputs = tf.constant(inputs, dtype=tf.float32)
cell = tf.nn.rnn_cell.LSTMCell(num_units = 1)
outputs ,states = tf.nn.dynamic_rnn(cell=cell, dtype = tf.float32, inputs=tf_inputs) 
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    outputs_run, states_run = sess.run([outputs, states])
    print(outputs_run)
    print(states_run.h) # 히든 states 값
    # 처음이기 때문에 출력값과 states값이 같다. 
    print(states_run.c) #c는 메모리 cell

[[[0.09927537]]]
[[0.09927537]]
[[0.18134572]]


## Dynamic_rnn이란?


#### 데이터 준비

In [38]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the thiadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]

df = pd.DataFrame(paragraph_dict_list)
df = df[["paragraph", "category"]]
df

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food
5,the garlic fries were a great starter (and a h...,food
6,our meal was excellent i had the pasta ai form...,food
7,what i enjoy most about palo alto is so many r...,food
8,the drinks came out fairly quickly a good two ...,food
9,despite the not so good burger the service was...,food


#### Preprocessing(데이터 전처리)

In [39]:
results = set() #  set : 순서없이 중복된 데이터가 없이 저장하는 변수 
df["paragraph"].str.lower().str.split().apply(results.update) 
# 전부 소문자로 바꿔서 공백을 기준으로 단어를 뽑아낸 다음 results에 저장 (set 에 저장하기 위해서 update사용 / 리스트의 append와 같은)

# 문장 분석을 위해서 단어들을 분리

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: paragraph, dtype: object

In [40]:
results

{'(and',
 '1',
 '1.5',
 '1.5x',
 '10-9',
 '14',
 '2',
 '2014',
 '33/1',
 'a',
 'about',
 'action',
 'after',
 'against',
 'ago',
 'ai',
 'alisson',
 'all',
 'almost',
 'also',
 'although',
 'alto',
 'amazing',
 'american',
 'and',
 'angelique',
 'any',
 'apps',
 'are',
 'area',
 'around',
 'as',
 'at',
 'australia',
 'avoided',
 'azarenka',
 'back',
 'battle',
 'be',
 'beef',
 'been',
 'before',
 'began',
 'began—too',
 'being',
 'better',
 'between',
 'big',
 'biscuits',
 'bit',
 'bookmark',
 'bookmarked',
 'both',
 'bottom',
 'brazilian',
 'bread',
 'breakthroughs',
 'briton',
 'brunch',
 'burger',
 'burgers',
 'business',
 'busy',
 'but',
 'by',
 'came',
 'campaigns',
 'can',
 'card',
 'caroline',
 'case',
 'certainly',
 'champion',
 'champions',
 'chance',
 'chances',
 'chicken',
 'city',
 'clay',
 'clean',
 'commented',
 'competition',
 'complain',
 'confidence',
 'contrast',
 'could',
 'course',
 'courteous',
 'courts',
 'coworkers',
 'crisp',
 'croatia',
 'crumbs',
 'culminating

In [41]:
# 단어들에게 일련번호를 붙여주는 작업
idx2word = dict(enumerate(results))

In [42]:
print(idx2word[369])
print(idx2word[396])

jose
years


In [43]:
# 
word2idx = {v:k for k, v in idx2word.items()}

In [44]:
print(word2idx["years"])
print(word2idx["jose"])

396
369


In [45]:
def encoded_paragraph(para):
    words = para.split()
    encoded = []
    for w in words :
        encoded.append([word2idx[w]])
    return encoded
# category 를 숫자로 
def encoded_category(cate):
    if cate == "food":
        return [1, 0]
    else:
        return [0, 1]

# 문장들의 길이가 다르기 떄문에 길이를 맞춰주어야 한다.     
# 가장 긴 문장을 기준으로 길이가 모자란 애들은 크기를 맞추기 위해 의미없는 값을 넣어 주어야 한다. 
def word_cnt(para):
    return len(para.split())

In [46]:
df["enc_paragraph"] = df.paragraph.apply(encoded_paragraph)
df.head()

,paragraph,category,enc_paragraph
0,dishplace is located in sunnyvale downtown the...,food,"[[223], [463], [460], [461], [276], [335], [16..."
1,service can be slower during busy hours but ou...,food,"[[345], [457], [387], [441], [46], [2], [329],..."
2,portions are huge both french toast and their ...,food,"[[62], [3], [60], [491], [289], [414], [222], ..."
3,we started with apps going the chicken and waf...,food,"[[45], [365], [458], [11], [353], [311], [281]..."
4,the biscuits and gravy was too salty two peopl...,food,"[[311], [316], [222], [338], [352], [156], [47..."


In [47]:
df["enc_category"] = df.category.apply(encoded_category)
df.head()

,paragraph,category,enc_paragraph,enc_category
0,dishplace is located in sunnyvale downtown the...,food,"[[223], [463], [460], [461], [276], [335], [16...","[1, 0]"
1,service can be slower during busy hours but ou...,food,"[[345], [457], [387], [441], [46], [2], [329],...","[1, 0]"
2,portions are huge both french toast and their ...,food,"[[62], [3], [60], [491], [289], [414], [222], ...","[1, 0]"
3,we started with apps going the chicken and waf...,food,"[[45], [365], [458], [11], [353], [311], [281]...","[1, 0]"
4,the biscuits and gravy was too salty two peopl...,food,"[[311], [316], [222], [338], [352], [156], [47...","[1, 0]"


In [48]:
df["seq_length"] = df.paragraph.apply(word_cnt)
df.head()

,paragraph,category,enc_paragraph,enc_category,seq_length
0,dishplace is located in sunnyvale downtown the...,food,"[[223], [463], [460], [461], [276], [335], [16...","[1, 0]",53
1,service can be slower during busy hours but ou...,food,"[[345], [457], [387], [441], [46], [2], [329],...","[1, 0]",19
2,portions are huge both french toast and their ...,food,"[[62], [3], [60], [491], [289], [414], [222], ...","[1, 0]",42
3,we started with apps going the chicken and waf...,food,"[[45], [365], [458], [11], [353], [311], [281]...","[1, 0]",43
4,the biscuits and gravy was too salty two peopl...,food,"[[311], [316], [222], [338], [352], [156], [47...","[1, 0]",82


In [55]:
# 문장의 최대 길이 알아내기
# df.sort_values(by=["seq_length"], axis=0, ascending = False)
max_word_cnt = 0

for row in df["paragraph"]:
    if len(row.split()) > max_word_cnt : 
        max_word_cnt = len(row.split())

print(max_word_cnt)

91


In [57]:
# 문장의 길이가 91개가 안될 경우 : -1 로 채우기 
def sequence_padding(enc_para):
    seq_len = len(enc_para)
    for i in range(seq_len, max_word_cnt):
        enc_para.append([-1])
    return enc_para

In [59]:
df["enc_paragraph"] = df.enc_paragraph.apply(sequence_padding)

In [60]:
print(df["enc_paragraph"][0])

[[223], [463], [460], [461], [276], [335], [165], [463], [64], [462], [311], [239], [471], [508], [457], [387], [438], [203], [364], [46], [53], [101], [329], [402], [490], [222], [140], [43], [203], [142], [454], [122], [427], [37], [31], [433], [524], [380], [125], [2], [512], [140], [501], [291], [213], [395], [82], [298], [451], [311], [262], [203], [480], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]


In [61]:
# 입력값들을 배열로 변환 
enc_paragraph = np.array(df.enc_paragraph.tolist())
enc_category = np.array(df.enc_category.tolist())
seq_length = np.array(df.seq_length.tolist())

In [70]:
seq_length

array([53, 19, 42, 43, 82, 24, 50, 43, 49, 82, 65, 88, 91, 71, 70, 30, 35,
       30, 63, 46])

#### 모델 구축

In [68]:
train_X = enc_paragraph
train_y = enc_category
print(train_X.shape, train_y.shape)

(20, 91, 1) (20, 2)


In [82]:
tf.reset_default_graph()
tf.set_random_seed(777)

# parameter 
lr = 0.001
total_epochs = 300

# mnay to one 
X = tf.placeholder(tf.float32, shape = [None, max_word_cnt, 1]) # 입력갯수는 91개 = max_word_cnt 
y = tf.placeholder(tf.int32, shape = [None, 2]) # food, sport 둘중하나이므로 2 

embeded = tf.layers.dense(X, 5)  # 학습과정을 좀 더 deep 하게 가져갈 때에도 dense를 통해 간편하게 만들 수 있다.

cell = tf.nn.rnn_cell.LSTMCell(num_units=64)
outputs ,state = tf.nn.dynamic_rnn(cell=cell, inputs=embeded, dtype = tf.float32, sequence_length= seq_length) 

# -1은 길이를 맞춰주기 위해 넣은 것이기 때문에 학습하면 안된다 
# 유효한 데이터만 가지고 학습할 수 있도록 해주는 것이 dynamic_rnn
# 융통성 있게 학습할 수 있게 해주는 게 dynamic_rnn
# 반대되는 것이 static_rnn으로 이는 고정된 길이만 학습할 수 있다. 
# 입력되는 갯수가 다르다는 것을 알려주기 위해서 sequence_length 인자를 사용

# state를 가져온 이유는 이중분류이기 때문에 outputs값은 사용하지않고 최종 히든스테이트를 통해 얻은 이중분류 값을 가져오게된다.
# state는 이미 평면화 작업이 되어있는 것이기 때문에 따로 평면화 작업을 할 필요는 없다. 
first_layer = tf.layers.dense(state.h, 32) 
logit = tf.layers.dense(first_layer, 2) #logit / 마지막 출력갯수는 맞춰주어야 한다.

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels= y))
train = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)

# print(X)
# print(embeded)
# print(state)
# print(first_layer)
# print(logit)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(total_epochs):
        _, c = sess.run([train,cost], feed_dict={X:train_X, y:train_y})
        if epoch % 50 == 0:
            pred = tf.nn.softmax(logit)
            correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, "float"))
            cur_acc = accuracy.eval({X:train_X, y:train_y})
            print("epoch:" + str(epoch) + ", cost:"+ str(c) + ", acc:" + str(cur_acc))

epoch:0, cost:0.8848035, acc:0.5
epoch:50, cost:0.40894026, acc:0.8
epoch:100, cost:0.0679994, acc:1.0
epoch:150, cost:0.003567428, acc:1.0
epoch:200, cost:0.0014474727, acc:1.0
epoch:250, cost:0.0008162061, acc:1.0


In [86]:
# https://insightcampus.co.kr/insightcommunity/?mod=document&uid=12936
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

tf.reset_default_graph()

start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


# Target log path
logs_path = 'log_dir/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# Text file containing words for training
training_file = 'data/belling_the_cat.txt'

def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    
    return content

training_data = read_data(training_file)
print("Loaded training data...")

def build_dataset(words): # 단어하고 숫자를 하나의 딕트형식으로 묶은 것
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)

    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary 
        # 키 : 값          값 : 키
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")

Loaded training data...
Iter= 1000, Average Loss= 4.334711, Average Accuracy= 4.80%
['spoke', '.', 'then'] - [the] vs [an]
Iter= 2000, Average Loss= 2.850103, Average Accuracy= 20.50%
['well', ',', 'but'] - [who] vs [,]
Iter= 3000, Average Loss= 2.593837, Average Accuracy= 30.50%
['was', 'in', 'the'] - [neighbourhood] vs [neighbourhood]
Iter= 4000, Average Loss= 2.097472, Average Accuracy= 45.70%
['when', 'she', 'was'] - [about] vs [about]
Iter= 5000, Average Loss= 1.978992, Average Accuracy= 50.10%
['be', 'procured', ','] - [and] vs [then]
Iter= 6000, Average Loss= 1.758271, Average Accuracy= 58.70%
['easily', 'escape', 'from'] - [her] vs [her]
Iter= 7000, Average Loss= 1.410386, Average Accuracy= 62.30%
['now', ',', 'if'] - [we] vs [,]
Iter= 8000, Average Loss= 1.311625, Average Accuracy= 64.60%
['that', 'our', 'chief'] - [danger] vs [the]
Iter= 9000, Average Loss= 1.167619, Average Accuracy= 68.80%
['thought', 'would', 'meet'] - [the] vs [in]
Iter= 10000, Average Loss= 1.073276, Ave

KeyboardInterrupt: Interrupted by user